In [2]:
import os, subprocess, json
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

class DenseEncoder(object):
        
        def xlm_roberta_base_encoder(self, input_text):
                # Load the tokenizer and model
                tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
                model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
                tokens = tokenizer.encode(input_text, add_special_tokens=True, max_length=512, truncation=True)
                input_ids = torch.tensor([tokens])
                with torch.no_grad():
                        outputs = model(input_ids)
                        dense_vector = outputs[0][0][0]  # Extract the dense vector from the model output

                # Convert the dense vector to a numpy array
                dense_vector_list = dense_vector.numpy().tolist()
                return dense_vector_list

        
        def document_encoder(self):
                datasets_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/datasets"
                encodes_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes"
                input_file_path = os.path.join(datasets_path, "mls-tasks-0/tasks.json")
                output_dir_path = os.path.join(encodes_path, "dense_encoder")
                
                with open(input_file_path) as f:
                        data = json.load(f)


                for d in data:
                        d["@id"] = d["id"]
                        d["id"] = int(d["id"].split("/")[-1])
                        d["vector"] = self.xlm_roberta_base_encoder(d["contents"])
                        d["contents"] = d["contents"] + "\n"

                        with open(f"{output_dir_path}/task.jsonl", "w") as jsonl_file:
                                for d in data:
                                        json.dump(d, jsonl_file)
                                        jsonl_file.write('\n')
                return


In [3]:
# from .logic import DenseEncoder

dense_enc = DenseEncoder()

dense_vec = dense_enc.xlm_roberta_base_encoder("Was is Elektrotechnik")

print(dense_vec)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0.07217977941036224, 0.09157858043909073, 0.05999606475234032, -0.020205063745379448, 0.02170461416244507, -0.02944577857851982, 0.03564104810357094, 0.0004519230860751122, 0.06526193022727966, -0.11502823233604431, 0.011782891117036343, 0.06713750213384628, -0.06128199025988579, 0.006231870502233505, 0.004843269940465689, 0.051830656826496124, -0.03202732279896736, -0.01102643646299839, 0.08238693326711655, 0.058150727301836014, 0.028431113809347153, -0.005219425540417433, 0.08151587843894958, 0.04477493464946747, -0.013574089854955673, 0.03810984641313553, -0.05324321985244751, 0.05443398654460907, 0.06877885013818741, 0.013043617829680443, 0.09326209872961044, -0.015830939635634422, 0.04249339923262596, 0.07140932977199554, 0.07173920422792435, 0.017670203000307083, -0.112077496945858, -0.07562334090471268, 0.059521548449993134, 0.07727187126874924, 0.07298506796360016, 0.11304982006549835, 0.025053592398762703, -0.03082236647605896, 0.019926216453313828, -0.07809607684612274, -0.0

In [4]:
import os

dense_enc.document_encoder()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are ini

In [6]:
import subprocess

cmd2 = """
    python -m pyserini.index.faiss \
        --input /home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes/dense_encoder \
        --output /home/peng_luh/__github/search_l3s/search_l3s_search_srv/indexes/dense/hnsw \
        --hnsw
"""

subprocess.call(cmd2, shell=True)

100%|██████████| 1/1 [00:00<00:00,  9.17it/s]


(326, 768)
hnsw_add_vertices: adding 326 elements on top of 0 (preset_levels=0)
  max_level = 0
Adding 326 elements at level 0
Done in 21.725 ms
326


0

In [2]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

folder_path = "/home/peng_luh/__github/search_l3s/search_l3s_search_srv/indexes/bm25/mls-tasks-1"

# Get the size of the folder in bytes
folder_size = get_folder_size(folder_path)

# Convert to appropriate units (KB, MB, GB, etc.)
folder_size_kb = folder_size / 1024
folder_size_mb = folder_size_kb / 1024
folder_size_gb = folder_size_mb / 1024

# Print the folder size
print(f"Folder size: {folder_size} bytes ({folder_size_kb:.2f} KB, {folder_size_mb:.2f} MB, {folder_size_gb:.2f} GB)")

Folder size: 9592 bytes (9.37 KB, 0.01 MB, 0.00 GB)


In [1]:
import os

if os.path.exists("/home/peng_luh/__github/search_l3s/search_l3s_search_srv/encodes/dense/xlm_roberta_base/mls-tasks-0/data_encoded.jsonl"):
    print("True")
else:
    print("False")

True
